<a href="https://colab.research.google.com/github/elixir-biohackathon-project-12/network-analysis/blob/main/get_collector_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check for gender

## Init and load data

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('id_list').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

# SPARQLWrapper
!pip install -q SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON

## Loop over de wd IDs

In [ ]:
for row in rows:
  url = str(row[0])
  name = 'unknown'
  gender = 'unknown'
  country = 'unknown'
  birth = 'S.D.'
  dead = 'S.D.'
  if 'http://www.wikidata.org/entity/' in url:
    item = url.replace('http://www.wikidata.org/entity/', 'wd:')
    s_query = """
    SELECT ?label ?gender ?genderLabel ?birthdate ?deaddate ?country ?countryLabel WHERE {{
      {0} rdfs:label ?label .
      OPTIONAL{{ {0} wdt:P21 ?gender .}}.
      OPTIONAL{{ {0} wdt:P569 ?birthdate .}}.
      OPTIONAL{{ {0} wdt:P570 ?deaddate .}}.
      OPTIONAL{{ {0} wdt:P27 ?country .}}.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}

    """.format(item)
    sparql = SPARQLWrapper('https://query.wikidata.org/sparql')
    sparql.setQuery(s_query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()
    try:
      result = results["results"]["bindings"][0]
      name = result['label']['value']
      try:
        gender = result['genderLabel']['value']
      except KeyError:
        gender = 'unknown'
      try:
        country = result['countryLabel']['value']
      except KeyError:
        country = 'unknown'
      try:
        birth = result['birthdate']['value']
      except KeyError:
        birth = 'S.D.'
      try:
        dead = result['deaddate']['value']
      except KeyError:
        dead = 'S.D.'
    except IndexError:
      pass

  else:
    orcid = url.replace('https://orcid.org/', '')
    s_query = """
    SELECT ?item ?itemLabel ?gender ?genderLabel ?birthdate ?deaddate ?country ?countryLabel WHERE {{
      ?item wdt:P496 '{0}' .
      OPTIONAL{{ ?item wdt:P21 ?gender .}}.
      OPTIONAL{{ ?item wdt:P569 ?birthdate .}}.
      OPTIONAL{{ ?item wdt:P570 ?deaddate .}}.
      OPTIONAL{{ ?item wdt:P27 ?country .}}.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """.format(orcid)
    sparql = SPARQLWrapper('https://query.wikidata.org/sparql')
    sparql.setQuery(s_query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()
    try:
      result = results["results"]["bindings"][0]
      name = result['itemLabel']['value']
      try:
        gender = result['genderLabel']['value']
      except KeyError:
        gender = 'unknown'
      try:
        country = result['countryLabel']['value']
      except KeyError:
        country = 'unknown'
      try:
        birth = result['birthdate']['value']
      except KeyError:
        birth = 'S.D.'
      try:
        dead = result['deaddate']['value']
      except KeyError:
        dead = 'S.D.'
    except IndexError:
      pass

  row.append(name)
  row.append(gender)
  row.append(country)
  row.append(birth)
  row.append(dead)




print(rows)

In [ ]:
df_2 = pd.DataFrame.from_records(rows)

print(df_2)

In [15]:
from gspread_dataframe import set_with_dataframe
title = 'collectors_info_v4'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, df_2) 